# Regrid monthly AWAP to model res

In [1]:
import sys
sys.path.append('../')  # import functions to make life easier
import climate_xr_funcs
import climate_droughts_xr_funcs

# import things
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime
import regionmask
import os
# import salem
from dask.diagnostics import ProgressBar
import xesmf as xe
from scipy import stats


# %% set file path to model output
filepath = '/Volumes/LaCie/CMIP5-PMIP3/CESM-LME/mon/PRECT_v6/'
filepath_pmip3 = '/Volumes/LaCie/CMIP5-PMIP3'
filepath_cesm_mon = '/Volumes/LaCie/CMIP5-PMIP3/CESM-LME/mon'

In [2]:
awap = xr.open_dataset('/Volumes/LaCie/droughts/Monthly_total_precipitation_AWAP_1900_2016_gapfilled_with_mon_mean.nc')
awap_masked = xr.open_dataset('/Volumes/LaCie/droughts/Monthly_total_precipitation_AWAP_masked_1900_2016.nc')

In [3]:
# rename awap things to be the same as the masked version
awap = awap.rename({'longitude': 'lon', 'latitude': 'lat', 'z':'time', 'variable': 'pre'})

# replace times with those in masked
awap['time'] = awap_masked.time
awap['PRECT_mm'] = awap.pre

In [4]:
awap

<xarray.Dataset>
Dimensions:   (lon: 886, lat: 691, time: 1404)
Coordinates:
  * lon       (lon) float64 112.0 112.0 112.1 112.1 ... 156.1 156.2 156.2 156.2
  * lat       (lat) float64 -10.0 -10.05 -10.1 -10.15 ... -44.4 -44.45 -44.5
  * time      (time) datetime64[ns] 1900-01-16 ... 2016-12-16
Data variables:
    pre       (time, lat, lon) float32 ...
    PRECT_mm  (time, lat, lon) float32 ...
Attributes:
    crs:          +proj=longlat +datum=WGS84
    crs_format:   PROJ.4
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 2.5-8)
    date:         2019-06-06 09:47:35

In [5]:
hist_output_dir = '../files/historical_1900'

In [6]:
bcc_precip_hist_annual_aus        = xr.open_dataset('%s/aus/bcc_precip_hist_annual_aus.nc' % hist_output_dir)
ccsm4_precip_hist_annual_aus      = xr.open_dataset('%s/aus/ccsm4_precip_hist_annual_aus.nc' % hist_output_dir)
csiro_mk3l_precip_hist_annual_aus = xr.open_dataset('%s/aus/csiro_mk3l_precip_hist_annual_aus.nc' % hist_output_dir)
fgoals_gl_precip_hist_annual_aus  = xr.open_dataset('%s/aus/fgoals_gl_precip_hist_annual_aus.nc' % hist_output_dir)
fgoals_s2_precip_hist_annual_aus  = xr.open_dataset('%s/aus/fgoals_s2_precip_hist_annual_aus.nc' % hist_output_dir)
giss_21_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_21_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_22_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_22_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_23_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_23_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_24_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_24_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_25_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_25_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_26_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_26_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_27_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_27_precip_hist_annual_aus.nc' % hist_output_dir)
# # giss_28_precip_hist_annual_aus    = xr.open_dataset('%s/aus/giss_28_precip_hist_annual_aus.nc' % hist_output_dir)
hadcm3_precip_hist_annual_aus     = xr.open_dataset('%s/aus/hadcm3_precip_hist_annual_aus.nc' % hist_output_dir)
ipsl_precip_hist_annual_aus       = xr.open_dataset('%s/aus/ipsl_precip_hist_annual_aus.nc' % hist_output_dir)
miroc_precip_hist_annual_aus      = xr.open_dataset('%s/aus/miroc_precip_hist_annual_aus.nc' % hist_output_dir)
mpi_precip_hist_annual_aus        = xr.open_dataset('%s/aus/mpi_precip_hist_annual_aus.nc' % hist_output_dir)
mri_precip_hist_annual_aus        = xr.open_dataset('%s/aus/mri_precip_hist_annual_aus.nc' % hist_output_dir)
# giss_all_precip_hist_annual_aus = xr.open_dataset('%s/aus/giss_all_precip_hist_annual_aus.nc' % hist_output_dir)
ff1_precip_hist_annual_aus = xr.open_dataset('%s/aus/cesmlme-ff1_precip_hist_annual_aus.nc' % hist_output_dir)

In [7]:
def regrid_files(ds_to_regrid, ds_target ):
    # resolution of output: same as cesm-lme
    ds_out = xr.Dataset({'lat': (['lat'], ds_target.lat.data),
                         'lon': (['lon'], ds_target.lon.data)})

    regridder = xe.Regridder(ds_to_regrid, ds_out, 'bilinear')
    # regridder.clean_weight_file()

    ds_out = regridder(ds_to_regrid)
    for k in ds_to_regrid.data_vars:
        print(k, ds_out[k].equals(regridder(ds_to_regrid[k])))

    return ds_out

In [8]:
def save_netcdf_compression(ds, output_dir, filename):

    comp = dict(zlib=True, complevel=8)
    encoding = {var: comp for var in ds.data_vars}

    delayed_obj = ds.to_netcdf('%s/%s.nc' % (output_dir, filename), mode='w', compute=False, encoding=encoding)
    with ProgressBar():
        results = delayed_obj.compute()

In [9]:
awap_masked_annual_bcc_res = regrid_files(awap, bcc_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_bcc_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_bcc')

awap_masked_annual_ccsm4_res = regrid_files(awap, ccsm4_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_ccsm4_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_ccsm4')

awap_masked_annual_csiro_mk3l_res = regrid_files(awap, csiro_mk3l_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_csiro_mk3l_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_csiro_mk3l')

awap_masked_annual_fgoals_gl_res = regrid_files(awap, fgoals_gl_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_fgoals_gl_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_fgoals_gl')

awap_masked_annual_fgoals_s2_res = regrid_files(awap, fgoals_s2_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_fgoals_s2_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_fgoals_s2')

awap_masked_annual_giss_21_res = regrid_files(awap, giss_21_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_giss_21_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_giss')

awap_masked_annual_hadcm3_res = regrid_files(awap, hadcm3_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_hadcm3_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_hadcm3')

awap_masked_annual_ipsl_res = regrid_files(awap, ipsl_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_ipsl_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_ipsl')

awap_masked_annual_miroc_res = regrid_files(awap, miroc_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_miroc_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_miroc')

awap_masked_annual_mpi_res = regrid_files(awap, mpi_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_mpi_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_mpi')

awap_masked_annual_mri_res = regrid_files(awap, mri_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_mri_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_mri')

awap_masked_annual_ff1_res = regrid_files(awap, ff1_precip_hist_annual_aus)
save_netcdf_compression(awap_masked_annual_ff1_res, hist_output_dir + '/awap_regridded', 'awap_gf_regridded_to_cesmlme')

pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s
pre True
PRECT_mm True
[########################################] | 100% Completed |  0.1s